In [1]:
#importin required libraries
import pandas as pd
import random
import nltk
import contractions
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
import re
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to C:\Users\Wills
[nltk_data]     Michael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Wills
[nltk_data]     Michael\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
#load the dataset
df = pd.read_csv("data_career.csv")
df.head()

,Position,Long Description,Company Name,Exp Years,Primary Keyword,English Level,Published,Long Description_lang,id,__index_level_0__
0,10 + Blockchain Nodes / Masternodes to set up,*Requirements*\r\n\r\nWe're looking for a long...,MyCointainer,2y,Sysadmin,intermediate,2020-10-01T00:00:00+03:00,en,c0ca96e7-85df-50df-a64e-d934cd02a170,27461
1,10 .NET Developers (Middle and Senior level),"Greetings! My name is Maria, I am in urgent ne...",TechScout.tech,2y,.NET,intermediate,2022-03-01T00:00:00+02:00,en,64f4b7ea-36e4-5bdd-a8b1-185f32f7dc7f,27462
2,"10X Engineer (co-founder, #4 employee, USD 11-...",**Product**\r\nThe product is a live video cha...,Innoteka,5y,JavaScript,fluent,2021-07-01T00:00:00+03:00,en,b9a1303e-dd0c-5ed1-8f62-be2bc4c7da4f,27463
3,16 - Amazon Brand Manager,"Currently, TCM expanding its activities to Ukr...",FirstFive,2y,Marketing,upper,2022-01-01T00:00:00+02:00,en,99cb3f4a-9b4b-53d9-9a3b-bab2c22da346,27464
4,16 - Amazon Brand Manager,"Hello,\r\nWe, MIMIRB2B, are an outstaff compan...",MimirB2B,1y,Marketing,upper,2021-12-01T00:00:00+02:00,en,bc1419f7-28e2-582b-8d53-22e28b2f0210,27465


## Step 3: Data Preprocessing


## Clean and prepare the dataset

**• Remove duplicates and missing values.**

**• Combine skills and job description into a single text column for similarity matching.**
## Preprocess text:
**• Convert to lowercase.**

**• Remove punctuation, numbers, and stopwords.**

**• (Optional) Lemmatization using NLTK or SpaCy**

## Clean and prepare the dataset


In [3]:
df.head(5)

,Position,Long Description,Company Name,Exp Years,Primary Keyword,English Level,Published,Long Description_lang,id,__index_level_0__
0,10 + Blockchain Nodes / Masternodes to set up,*Requirements*\r\n\r\nWe're looking for a long...,MyCointainer,2y,Sysadmin,intermediate,2020-10-01T00:00:00+03:00,en,c0ca96e7-85df-50df-a64e-d934cd02a170,27461
1,10 .NET Developers (Middle and Senior level),"Greetings! My name is Maria, I am in urgent ne...",TechScout.tech,2y,.NET,intermediate,2022-03-01T00:00:00+02:00,en,64f4b7ea-36e4-5bdd-a8b1-185f32f7dc7f,27462
2,"10X Engineer (co-founder, #4 employee, USD 11-...",**Product**\r\nThe product is a live video cha...,Innoteka,5y,JavaScript,fluent,2021-07-01T00:00:00+03:00,en,b9a1303e-dd0c-5ed1-8f62-be2bc4c7da4f,27463
3,16 - Amazon Brand Manager,"Currently, TCM expanding its activities to Ukr...",FirstFive,2y,Marketing,upper,2022-01-01T00:00:00+02:00,en,99cb3f4a-9b4b-53d9-9a3b-bab2c22da346,27464
4,16 - Amazon Brand Manager,"Hello,\r\nWe, MIMIRB2B, are an outstaff compan...",MimirB2B,1y,Marketing,upper,2021-12-01T00:00:00+02:00,en,bc1419f7-28e2-582b-8d53-22e28b2f0210,27465


In [4]:
#Identifying the features we would use for this project and dropping irrelevant features
df = df.drop(['Company Name', 'Exp Years',
             'English Level', 'Published',
       'Long Description_lang', 'id', '__index_level_0__', 'Primary Keyword'], axis=1)

In [5]:
#Rename relevant columns to match project needs
df = df.rename(columns={
    'Position': 'Job_Title',
    'Long Description': 'Job_Description'
})

## Remove duplicates and missing values

In [6]:
#checking for missing values
df.isnull().sum()

Job_Title          0
Job_Description    0
dtype: int64

In [7]:
#checking for duplicates
df.duplicated().sum()

0

In [8]:
df.head()

,Job_Title,Job_Description
0,10 + Blockchain Nodes / Masternodes to set up,*Requirements*\r\n\r\nWe're looking for a long...
1,10 .NET Developers (Middle and Senior level),"Greetings! My name is Maria, I am in urgent ne..."
2,"10X Engineer (co-founder, #4 employee, USD 11-...",**Product**\r\nThe product is a live video cha...
3,16 - Amazon Brand Manager,"Currently, TCM expanding its activities to Ukr..."
4,16 - Amazon Brand Manager,"Hello,\r\nWe, MIMIRB2B, are an outstaff compan..."


## Cleaning some Basic Inconsistencies from the dataset

In [9]:
#cleaning the job_title colum
def clean_job_title(title):
    """
    Cleans raw job title text by removing numbers, symbols, brackets, and non-English characters.
    """
    title = re.sub(r'^[^A-Za-z]+', '', str(title))      # Remove leading numbers/symbols
    title = re.sub(r'[\(\[].*?[\)\]]', '', title)       # Remove content inside () or []
    title = re.sub(r'[^A-Za-z0-9\s\.-]', '', title)     # Remove non-English chars
    title = re.sub(r'\s+', ' ', title).strip()          # Remove extra spaces
    return title
    
#Applying the clean job_title
df["Job_Title_Clean"] = df["Job_Title"].apply(clean_job_title)

In [10]:
#Viewing the job_titlr column to see the changes
check_mapping1 = df.groupby("Job_Title")["Job_Title_Clean"].unique().reset_index()

# show first few rows after the changes
check_mapping1.sample(10)

,Job_Title,Job_Title_Clean
30444,Lead Python developer for Risk Management Plat...,[Lead Python developer for Risk Management Pla...
3689,Automation QA Engineer (Cypress),[Automation QA Engineer]
58106,Senior Performance and Reliability Engineer in...,[Senior Performance and Reliability Engineer i...
1956,Administrative Worker,[Administrative Worker]
47180,RPA (Robotic Process Automation - UI Path/Python),[RPA]
63399,Sr. C# .Net/Core developer (Fintech project),[Sr. C .NetCore developer]
1800,Account manager (Italian language),[Account manager]
53987,Senior FE engineer,[Senior FE engineer]
30490,Lead QA Platforms,[Lead QA Platforms]
28389,Junior to Middle UI/UX Designer with interest ...,[Junior to Middle UIUX Designer with interest ...


In [11]:
#checking the job decription column to find the cleaning procedures
df["Job_Description"].value_counts()

Job_Description
*Requirements*\r\n\r\nWe're looking for a long term collaboration with someone that has an experience in crypto, masternodes, nodes, validators etc. We need to set up:\r\n\r\nKyber Network\r\nNebulas\r\nSecretNetwork\r\nTron\r\nAion\r\nDeFiChain\r\nEOS\r\nTomoChain\r\nElrond\r\nIRISnet\r\nIoTeX\r\nTerra\r\nChainX\r\nThorchain\r\n\r\nSuccesful candidates will have an opportunity to get more jobs and long term collaboration.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [12]:
#cleaning the job_title colum
def clean_job_description(text):
    """
    Cleans raw job description text by removing numbers, symbols, brackets, and non-English characters.
    """
    text = re.sub(r'^[^A-Za-z]+', '', str(text))      # Remove leading numbers/symbols
    text = re.sub(r"(\*{1,2}|•|✔|#)", " ", text)     # Remove markdown symbols and bullets
    text = re.sub(r'[\(\[].*?[\)\]]', '', text)       # Remove content inside () or []
    text = re.sub(r'[^A-Za-z0-9\s\.-]', '', text)     # Remove non-English chars
    text = re.sub(r'\s+', ' ', text).strip()          # Remove extra spaces
    return text
    
#Applying the clean job_title
df["Job_Description_Clean"] = df["Job_Description"].apply(clean_job_description)

In [13]:
df.head()

,Job_Title,Job_Description,Job_Title_Clean,Job_Description_Clean
0,10 + Blockchain Nodes / Masternodes to set up,*Requirements*\r\n\r\nWe're looking for a long...,Blockchain Nodes Masternodes to set up,Requirements Were looking for a long term coll...
1,10 .NET Developers (Middle and Senior level),"Greetings! My name is Maria, I am in urgent ne...",NET Developers,Greetings My name is Maria I am in urgent need...
2,"10X Engineer (co-founder, #4 employee, USD 11-...",**Product**\r\nThe product is a live video cha...,X Engineer,Product The product is a live video chat and c...
3,16 - Amazon Brand Manager,"Currently, TCM expanding its activities to Ukr...",Amazon Brand Manager,Currently TCM expanding its activities to Ukra...
4,16 - Amazon Brand Manager,"Hello,\r\nWe, MIMIRB2B, are an outstaff compan...",Amazon Brand Manager,Hello We MIMIRB2B are an outstaff company and ...


In [14]:
# checking compirirng the changes for the both columns
# sample = df.sample(20, random_state=42)[["Job_Title", "Job_Title_Clean", "Job_Description", "Job_Description_Clean"]]
# print(sample.to_string())

In [15]:
df.head()

,Job_Title,Job_Description,Job_Title_Clean,Job_Description_Clean
0,10 + Blockchain Nodes / Masternodes to set up,*Requirements*\r\n\r\nWe're looking for a long...,Blockchain Nodes Masternodes to set up,Requirements Were looking for a long term coll...
1,10 .NET Developers (Middle and Senior level),"Greetings! My name is Maria, I am in urgent ne...",NET Developers,Greetings My name is Maria I am in urgent need...
2,"10X Engineer (co-founder, #4 employee, USD 11-...",**Product**\r\nThe product is a live video cha...,X Engineer,Product The product is a live video chat and c...
3,16 - Amazon Brand Manager,"Currently, TCM expanding its activities to Ukr...",Amazon Brand Manager,Currently TCM expanding its activities to Ukra...
4,16 - Amazon Brand Manager,"Hello,\r\nWe, MIMIRB2B, are an outstaff compan...",Amazon Brand Manager,Hello We MIMIRB2B are an outstaff company and ...


In [16]:
#Droping the unwanted features in our dataframe
df =df.drop(["Job_Title", "Job_Description"], axis=1)

In [17]:
#Rename the columns to match project needs
df = df.rename(columns={
    'Job_Title_Clean': 'Job_Title',
    'Job_Description_Clean': 'Job_Description'
})

In [18]:
df.head()

,Job_Title,Job_Description
0,Blockchain Nodes Masternodes to set up,Requirements Were looking for a long term coll...
1,NET Developers,Greetings My name is Maria I am in urgent need...
2,X Engineer,Product The product is a live video chat and c...
3,Amazon Brand Manager,Currently TCM expanding its activities to Ukra...
4,Amazon Brand Manager,Hello We MIMIRB2B are an outstaff company and ...


In [19]:
# Count rows with empty strings in Job_Title
empty_titles = (df['Job_Title'].str.strip() == "").sum()
print("Empty job titles:", empty_titles)

Empty job titles: 26


In [20]:
# Remove rows where Job_Title is empty of just spaces
df = df[df['Job_Title'].str.strip() != ""]

# Reset index
df = df.reset_index(drop=True)

In [21]:
# Count rows with empty strings in Job_Title after effecting the changes
empty_titles = (df['Job_Title'].str.strip() == "").sum()
print("Empty job titles after the changes:", empty_titles)

Empty job titles after the changes: 0


In [22]:
df2=df

## Preprocessing text step
**• Convert to lowercase.**

**• Remove punctuation, numbers, and stopwords.**

**• (Optional) Lemmatization using NLTK or SpaCy**

In [23]:
sample = df2.sample(20, random_state=42)[["Job_Title", "Job_Description"]]
print(sample.to_string())

                                                                        Job_Title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [24]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


def clean_text(text):
    """
    Cleans input text for NLP tasks by:
    1. Converting text to lowercase
    2. Removing punctuation and numbers
    3. Tokenizing into words
    4. Removing stopwords
    5. Applying lemmatization
    6. Put everything back into single cleaned string
    """

    # ---------- Step 1: Lowercase ----------
    text = text.lower()

    # ---------- Step 2: Remove punctuation & numbers ----------
    text = re.sub(r'[^a-z\s]', '', text)   # keep only letters and spaces

    # ---------- Step 3: Tokenize ----------
    tokens = word_tokenize(text)

    # ---------- Step 4: Remove stopwords ----------
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # ---------- Step 5: Lemmatization ----------
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # ---------- Step 6: Rejoin ----------
    return " ".join(tokens)

In [25]:
import nltk
nltk.download('punkt')

# Applying my function
df2["Job_Title_Clean_text"] = df2["Job_Title"].apply(clean_text)
df2["Job_Description_Clean_text"] = df2["Job_Description"].apply(clean_text)

[nltk_data] Downloading package punkt to C:\Users\Wills
[nltk_data]     Michael\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
df2.head()

,Job_Title,Job_Description,Job_Title_Clean_text,Job_Description_Clean_text
0,Blockchain Nodes Masternodes to set up,Requirements Were looking for a long term coll...,blockchain node masternodes set,requirement looking long term collaboration so...
1,NET Developers,Greetings My name is Maria I am in urgent need...,net developer,greeting name maria urgent need find team net ...
2,X Engineer,Product The product is a live video chat and c...,x engineer,product product live video chat cobrowsing web...
3,Amazon Brand Manager,Currently TCM expanding its activities to Ukra...,amazon brand manager,currently tcm expanding activity ukraine hirin...
4,Amazon Brand Manager,Hello We MIMIRB2B are an outstaff company and ...,amazon brand manager,hello mimirbb outstaff company looking experie...


In [27]:
# Checking the changes observed on both columns
sample2 = df2.sample(10, random_state=42)[["Job_Title", "Job_Title_Clean_text", "Job_Description", "Job_Description_Clean_text"]]
print(sample2.to_string())

                                                                        Job_Title                                                Job_Title_Clean_text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [28]:
# Keep only the cleaned columns
df2 = df2[["Job_Title_Clean_text", "Job_Description_Clean_text"]]

# Rename them to shorter names for convenience
df2 = df2.rename(columns={
   "Job_Title_Clean_text": "Job_Title_Clean",
    "Job_Description_Clean_text": "Job_Description_Clean"})

In [29]:
df2.head()

,Job_Title_Clean,Job_Description_Clean
0,blockchain node masternodes set,requirement looking long term collaboration so...
1,net developer,greeting name maria urgent need find team net ...
2,x engineer,product product live video chat cobrowsing web...
3,amazon brand manager,currently tcm expanding activity ukraine hirin...
4,amazon brand manager,hello mimirbb outstaff company looking experie...


 ## Step 4: Feature Extraction Step

#### Convert text into numerical features using: 

###### - TF-IDF Vectorizer (sklearn.feature_extraction.text.TfidfVectorizer) 

#### Create two sets of vectors: 

###### - For job roles (dataset) 
###### - For user input 

#### The First set of vectors: 
- Job roles (dataset) 

In [34]:
# Import the TF-IDF tool from sklearn
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Drop any rows where the description is missing (NaN)
# "tolist()" changes it from a pandas column into a normal python list
job_roles = df2["Job_Description_Clean"].dropna().tolist()

# Initialize TF-IDF Vectorizer to convert text into numerical features
# limit features to top 5000 words
# unigrams + bigrams capture short skills
# remove English stopwords

vectorizer = TfidfVectorizer(
    max_features=5000,   
    ngram_range=(1,2),   
    stop_words="english" 
)

X_roles = vectorizer.fit_transform(job_roles)

# Print the size of the TF-IDF matrix for job description
print("Job roles TF-IDF shape:", X_roles.shape)

Job roles TF-IDF shape: (141871, 5000)


#### The Second set of vectors: 
- User input 

In [31]:
# Convert user input
user_input = "Looking for a Java Developer with Spring Boot, SQL, and Docker experience."

# Transform user input (also sparse, same feature space)
X_user = vectorizer.transform([user_input])

print("User Input TF-IDF shape:", X_user.shape)


User Input TF-IDF shape: (1, 5000)


In [35]:
df2

,Job_Title_Clean,Job_Description_Clean
0,blockchain node masternodes set,requirement looking long term collaboration so...
1,net developer,greeting name maria urgent need find team net ...
2,x engineer,product product live video chat cobrowsing web...
3,amazon brand manager,currently tcm expanding activity ukraine hirin...
4,amazon brand manager,hello mimirbb outstaff company looking experie...
...,...,...
141866,gaming,cto lead development platform implementation n...
141867,lead full stack developer,required attribute full project lifecycle expe...
141868,saldo apps,saldo apps seeking seasoned cto lead team exce...
141869,lead react,responsibility analyze estimate implementation...
